In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-patient-precondition-dataset/covid.csv
/kaggle/input/covid19-patient-precondition-dataset/Description.xlsx
/kaggle/input/covid19-patient-precondition-dataset/Catalogs.xlsx


In [29]:
Df = pd.read_csv('/kaggle/input/covid19-patient-precondition-dataset/covid.csv')
Df

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2,1,04-05-2020,02-05-2020,9999-99-99,97,2,27,97,...,2,2,2,2,2,2,2,2,1,97
1,1009bf,2,1,19-03-2020,17-03-2020,9999-99-99,97,2,24,97,...,2,2,2,2,2,2,2,99,1,97
2,167386,1,2,06-04-2020,01-04-2020,9999-99-99,2,2,54,2,...,2,2,2,2,1,2,2,99,1,2
3,0b5948,2,2,17-04-2020,10-04-2020,9999-99-99,2,1,30,97,...,2,2,2,2,2,2,2,99,1,2
4,0d01b5,1,2,13-04-2020,13-04-2020,22-04-2020,2,2,60,2,...,2,1,2,1,2,2,2,99,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566597,01ff60,2,1,13-05-2020,03-05-2020,9999-99-99,97,2,58,97,...,2,2,2,2,2,2,2,2,3,97
566598,047cd1,1,1,07-04-2020,06-04-2020,9999-99-99,97,2,48,2,...,2,2,2,2,2,2,2,2,3,97
566599,1beb81,1,2,14-05-2020,01-05-2020,9999-99-99,2,1,49,2,...,2,2,2,2,1,2,2,99,3,2
566600,16fb02,1,1,31-05-2020,29-05-2020,9999-99-99,97,1,43,2,...,2,2,2,2,2,2,2,2,3,97


In [11]:
from datetime import datetime
datetime.strptime('2012-07-12', '%Y-%m-%d').timetuple()

time.struct_time(tm_year=2012, tm_mon=7, tm_mday=12, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=194, tm_isdst=-1)

In [36]:
Df['died'] = (Df['date_died'].astype('string') != '9999-99-99').astype(int)
Df['date_symptoms'] = pd.to_datetime(Df['date_symptoms'] ,format='%d-%m-%Y')
Df['entry_date'] = pd.to_datetime(Df['entry_date'],format='%d-%m-%Y')

Df['time_to_admission'] = (Df['entry_date'] - Df['date_symptoms']).dt.days

In [45]:
ord_cols = {'id','entry_date','date_symptoms','date_died','died','time_to_admission','age'}
cat_cols = []
for col in Df.columns:
    if col in ord_cols:
        continue
    cat_cols.append(col)
print(cat_cols)
df_ohe = pd.get_dummies(Df,columns=cat_cols,drop_first=True)

['sex', 'patient_type', 'intubed', 'pneumonia', 'pregnancy', 'diabetes', 'copd', 'asthma', 'inmsupr', 'hypertension', 'other_disease', 'cardiovascular', 'obesity', 'renal_chronic', 'tobacco', 'contact_other_covid', 'covid_res', 'icu']


In [49]:
df_ohe = df_ohe.drop(columns = ['id','entry_date','date_symptoms','date_died'])
df_ohe

,age,died,time_to_admission,sex_2,patient_type_2,intubed_2,intubed_97,intubed_99,pneumonia_2,pneumonia_99,...,renal_chronic_98,tobacco_2,tobacco_98,contact_other_covid_2,contact_other_covid_99,covid_res_2,covid_res_3,icu_2,icu_97,icu_99
0,27,0,2,1,0,0,1,0,1,0,...,0,1,0,1,0,0,0,0,1,0
1,24,0,2,1,0,0,1,0,1,0,...,0,1,0,0,1,0,0,0,1,0
2,54,0,5,0,1,1,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
3,30,0,7,1,1,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
4,60,1,0,0,1,1,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566597,58,0,10,1,0,0,1,0,1,0,...,0,1,0,1,0,0,1,0,1,0
566598,48,0,1,0,0,0,1,0,1,0,...,0,1,0,1,0,0,1,0,1,0
566599,49,0,13,0,1,1,0,0,0,0,...,0,1,0,0,1,0,1,1,0,0
566600,43,0,2,0,0,0,1,0,0,0,...,0,1,0,1,0,0,1,0,1,0


In [51]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(
    df_ohe.drop(columns = ['died']), df_ohe['died'], test_size=0.20, random_state=42)

In [57]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import confusion_matrix as cm
clf = LR()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
acc(y_test,y_pred)


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9426761147536644

In [59]:
from sklearn.metrics import confusion_matrix as cm

cm(y_test,y_pred)

array([[104377,   1647],
       [  4849,   2448]])